# 1. Load packages

In [1]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])
y_train_full = train_full_data_meta[['label']]
    
print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

[########################################] | 100% Completed |  7.7s
[########################################] | 100% Completed |  3.6s
- Number of train full data: 4277
- Number of test data: 2154


# 3. Training

In [5]:
from sklearn.preprocessing import OneHotEncoder

IMG_SIZE    = 512
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))
y_enc = OneHotEncoder(sparse=False, dtype=bool)
y_train_full = y_enc.fit_transform(y_train_full)
n_classes = len(y_enc.categories_[0])
print("- Number of classes:", n_classes)

- Number of classes: 88


In [6]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Fix pretrained model

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)
sample_weight_train = compute_sample_weight(class_weight='balanced', y=y_train.argmax(1))
sample_weight_val   = compute_sample_weight(class_weight='balanced', y=y_val.argmax(1))

train_ds = preprocess(tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weight_train)), True, BATCH_SIZE)
val_ds   = preprocess(tf.data.Dataset.from_tensor_slices((X_val, y_val, sample_weight_val)), False, BATCH_SIZE)
test_ds  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train.shape:", X_train.shape, y_train.shape)
print("- val.shape:", X_val.shape, y_val.shape)
print("- test.shape:", X_test.shape)

- train.shape: (3207, 512, 512, 3) (3207, 88)
- val.shape: (1070, 512, 512, 3) (1070, 88)
- test.shape: (2154, 512, 512, 3)


In [8]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    with strategy.scope():
        base_model = keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    return model, base_model

In [9]:
from analysis_tools.modeling import *

model, base_model = build_model(n_classes, strategy)
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=10, plot_path=join(PATH.result, 'proposed3', 'fix_pretrained_model')))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

Epoch 43/1000
101/101 [==============================] - 14s 99ms/step - loss: 0.5008 - f1_score: 0.7246 - val_loss: 1.0357 - val_f1_score: 0.5354
Epoch 44/1000
101/101 [==============================] - 14s 108ms/step - loss: 0.5069 - f1_score: 0.7441 - val_loss: 1.0355 - val_f1_score: 0.5706
Epoch 45/1000
101/101 [==============================] - 13s 100ms/step - loss: 0.5050 - f1_score: 0.7346 - val_loss: 1.0250 - val_f1_score: 0.5002
Epoch 46/1000
101/101 [==============================] - 14s 102ms/step - loss: 0.4773 - f1_score: 0.7399 - val_loss: 1.0803 - val_f1_score: 0.4952
Epoch 47/1000
101/101 [==============================] - 13s 103ms/step - loss: 0.4916 - f1_score: 0.7309 - val_loss: 1.1168 - val_f1_score: 0.5067
Epoch 48/1000
101/101 [==============================] - 14s 112ms/step - loss: 0.4867 - f1_score: 0.7307 - val_loss: 1.0167 - val_f1_score: 0.5581
Epoch 49/1000
101/101 [==============================] - 12s 99ms/step - loss: 0.4894 - f1_score: 0.7354 - val_lo

In [11]:
from dacon_submit_api.dacon_submit_api import post_submission_file

submission_file_path = join(PATH.output, 'proposed3_no_fine_tuning.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    'proposed3_no_fine_tuning'
)

{'isSubmitted': True, 'detail': 'Success'}


## 3.2 Fine-tuning

In [12]:
with strategy.scope():
    base_model.trainable = True
    model.compile(optimizer=keras.optimizers.Adam(2e-4), loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=10, plot_path=join(PATH.result, 'proposed3', 'fine_tuning')))

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
101/101 [==============================] - 81s 481ms/step - loss: 1.7682 - f1_score: 0.3840 - val_loss: 1.4050 - val_f1_score: 0.4436
Epoch 2/1000
101/101 [==============================] - 39s 357ms/step - loss: 1.0444 - f1_score: 0.5215 - val_loss: 1.6742 - val_f1_score: 0.4517
Epoch 3/1000
101/101 [==============================] - 39s 354ms/step - loss: 0.7984 - f1_score: 0.5914 - val_loss: 1.3156 - val_f1_score: 0.5347
Epoch 4/1000
101/101 [==============================] - 40s 363ms/step - loss: 0.6975 - f1_score: 0.6360 - val_loss: 1.3943 - val_f1_score: 0.5822
Epoch 5/1000
101/101 [==============================] - 39s 353ms/step - loss: 0.6744 - f1_score: 0.6618 - val_loss: 1.1686 - val_f1_score: 0.5656
Epoch 6/1000
101/101 [==============================] - 37s 352ms/step - loss: 0.5207 - f1_

# 4. Evaluation

In [13]:
submission_file_path = join(PATH.output, 'proposed3_fine_tuning.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)
submission

,label
index,
0,tile-glue_strip
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-good


# 5. Submission

In [14]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    'proposed3_fine_tuning'
)

{'isSubmitted': True, 'detail': 'Success'}
